In [1]:
import pandas as pd
import geopandas as gp
import os
import json
from datetime import datetime

In [2]:
df10 = pd.read_csv("Ref10_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df11 = pd.read_csv("Ref11_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df12 = pd.read_csv("Ref12_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df13 = pd.read_csv("Ref13_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df14 = pd.read_csv("Ref14_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )

####Filter by 100 Employees for PNATITL Grouping

In [3]:
df10 = df10[(df10["EMPSDT"] > 0) & (df10["EMPSDT"] < 100)]
df11 = df11[(df11["EMPSDT"] > 0) & (df11["EMPSDT"] < 100)]
df12 = df12[(df12["EMPSDT"] > 0) & (df12["EMPSDT"] < 100)]
df13 = df13[(df13["EMPSDT"] > 0) & (df13["EMPSDT"] < 100)]
df14 = df14[(df14["EMPSDT"] > 0) & (df14["EMPSDT"] < 100)]

In [4]:
files = [df10, df11, df12, df13, df14]

for df in files:
    df['count'] = 1

####Group by SICD Codes

In [5]:
df10_grouped = df10.groupby(['PRMSIC', 'PRMSICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

In [6]:
cols = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

####Group by PNATITL Codes

In [110]:
df10_grouped = df10.groupby(['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

In [111]:
cols = ['PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

####Append NAICS Labels (For Both SICD and PNATITL Groupings)

In [7]:
grouped_files = [df10_grouped, df11_grouped, df12_grouped, df13_grouped, df14_grouped]

In [8]:
def getNAICS(row):
    code = str(row['PNACODE'])[0:2]
    return code

In [9]:
def getNAICSLabel(row):
    label = naicsCODES[row['NAICS']] 
    return label

In [10]:
def createBoroCTCode(row):
    new_county_code = counties_dict[row['Geo_COUNTY']]
    grouping_code = new_county_code + row['SHP_CENSUS_TRACT']
    return grouping_code

In [11]:
naicsCODES = \
{'11': 'AGRICULTURE, FORESTRY, FISHING AND HUNTING',
 '21': 'MINING',
 '22': 'UTILITIES',
 '23': 'CONSTRUCTION',
 '31': 'MANUFACTURING',
 '32': 'MANUFACTURING',
 '33': 'MANUFACTURING',
 '42': 'WHOLESALE TRADE',
 '44': 'RETAIL TRADE',
 '45': 'RETAIL TRADE',
 '48': 'TRANSPORTATION AND WAREHOUSING',
 '49': 'TRANSPORTATION AND WAREHOUSING',
 '51': 'INFORMATION',
 '52': 'FINANCE AND INSURANCE',
 '53': 'REAL ESTATE RENTAL AND LEASING',
 '54': 'PROFESSIONAL, SCIENTIFIC, AND TECHNICAL SERVICES',
 '55': 'MANAGEMENT OF COMPANIES AND ENTERPRISES',
 '56': 'ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIATION SERVICES',
 '61': 'EDUCATIONAL SERVICES',
 '62': 'HEALTH CARE AND SOCIAL ASSISTANCE',
 '71': 'ARTS, ENTERTAINMENT, AND RECREATION',
 '72': 'ACCOMMODATION AND FOOD SERVICES',
 '81': 'OTHER SERVICES (EXCEPT PUBLIC ADMINISTRATION)',
 '92': 'PUBLIC ADMINISTRATION',
 '99': 'UNCLASSIFIED'}

In [12]:
counties_dict = {
    '085': '5',
    '005': '2',
    '047': '3',
    '061': '1',
    '081': '4'}

In [13]:
df_clusters = pd.read_csv("../Lodes/CTclusters_code.csv", index_col = False, dtype = {'code': object})
del df_clusters['Unnamed: 0']
# ls ../Lodes/

In [14]:
for df in grouped_files:
    df.columns = df.columns.droplevel(1)
    df.columns = cols 
    del df['count_MED']
    df.loc[:,'NAICS'] = df.apply(getNAICS, axis = 1)
    df.loc[:,'NAICS_LABEL'] = df.apply(getNAICSLabel, axis = 1)
    df.loc[:,'Grouping_Code'] = df.apply(createBoroCTCode, axis = 1)
    #df = pd.merge(df, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])

In [15]:
df10_grouped = pd.merge(df10_grouped, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])
df11_grouped = pd.merge(df11_grouped, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])
df12_grouped = pd.merge(df12_grouped, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])
df13_grouped = pd.merge(df13_grouped, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])
df14_grouped = pd.merge(df14_grouped, df_clusters[['cluster', 'code']], left_on = ['Grouping_Code'], right_on = ['code'])

In [16]:
df10_grouped.columns

Index([u'PRMSIC', u'SICD', u'PNACODE', u'PNATITL', u'Geo_COUNTY',
       u'SHP_BOROUGH', u'SHP_CENSUS_TRACT', u'lmi_ct', u'EMPSDT_SUM',
       u'EMPSDT_MED', u'SLSVDT_SUM', u'SLSVDT_MED', u'count_SUM', u'NAICS',
       u'NAICS_LABEL', u'Grouping_Code', u'cluster', u'code'],
      dtype='object')

####Upload Borough Shapefile Data

In [17]:
boros = gp.GeoDataFrame.from_file("../nyct2010_15b/nyct2010.shp")

####Append Shapefile Data

In [18]:
import shapefile
from pyproj import Proj, transform 
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon

In [19]:
def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x2,y2 = transform(inProj,outProj,x0,y0)
    
    return (x2, y2)

In [20]:
def convertCoordinates(row):
    geo = row['geometry']
    
    try: 
        x = list(geo.exterior.coords)
        new_list = [] 
    
        for i in x:
            new_list.append(convert(i))
        
        return Polygon(new_list)
    
    except: 
        x0 = list(geo[0].exterior.coords)
        x1 = list(geo[1].exterior.coords) 

        new_list0 = []
        new_list1 = []

        for i in x0:
            new_list0.append(convert(i))
    
        for j in x1:
            new_list1.append(convert(j))
    
        return MultiPolygon((Polygon(new_list0), Polygon(new_list1)))

####Create Directories with Files

In [21]:
#YEARS = ["2010", "2011", "2012", "2013", "2014"]
#NAICS = list(df.groupby(['NAICS_LABEL', 'PNATITL']).agg('count').index.get_level_values(0))
#PNATITL = list(df.groupby(['NAICS_LABEL', 'PNATITL']).agg('count').index.get_level_values(1))

YEARS = ["2010", "2011", "2012", "2013", "2014"]
NAICS = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(0))
PNATITL = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(1))
SICD = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(2))

In [22]:
cnt = len(PNATITL)
base_loc = '/gpfs1/cusp/kl1771/nycem/Ref_LMI/geojson'

In [23]:
for year in YEARS:
    year_folder = str(year)
    
    if year_folder in os.listdir(base_loc):
        print 'already created'
    else:
        add_year = base_loc + "/" + year_folder
        os.mkdir(add_year)

already created
already created
already created
already created
already created


# CHANGE this Sam! 

In [30]:
df = df14_grouped #change this df11_grouped, df12_grouped etc. 
year = "2014" # change this to 2011, 2012, 2013 etc, 

base_year_loc = base_loc + "/" + year
year_dir = os.listdir(base_year_loc)

In [31]:
def createGeoJSON(pnatitl_loc, string2):
    
    #pnatitl_loc = base_year_loc + "/" + string1
    sicd_code = string2.replace("/", "-")
    sicd_file_name = sicd_code.replace("_", "").replace(", ", "_").replace(" ", "_").replace("(", "").\
    replace(")", "").lower()
    

    df_temp = df[df['SICD'] == string2]
    
    result = pd.merge(df_temp, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
        
    result = result[['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS_LABEL', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
                 'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'NTAName', 'Shape_Area', 'geometry', 'cluster']]
    
    result.columns = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'NAICS', 'Borough', 'Census_Tract', \
                  'Median_Income_ct', 'EMPSDT_Sum', 'EMPSDT_Median', 'SLSVDT_Sum', 'SLSVDT_Median', \
                  'Business_Count', 'Neighborhood_Name', 'Shape_Area', 'geometry', 'cluster']
    
    result.loc[:,'geometry'] = result.apply(convertCoordinates, axis = 1)
    geojson_string = gp.GeoDataFrame(result).to_json()
    geojson_dict = json.loads( eojson_string)
    loc = pnatitl_loc + "/" + sicd_file_name + ".geojson"
    
    with open(loc, 'wb') as fp:
        json.dump(geojson_dict, fp)

In [32]:
startTime = datetime.now()

for index in range(cnt): 
    
    year_dir = os.listdir(base_year_loc)
    
    naics = NAICS[index]
    naics_dir_name = naics.replace("_", "").replace(", ", "_").replace(" ", "_").replace("(", "").\
    replace(")", "").lower()
    
    pnatitl = PNATITL[index]
    pnatitl_dir_name = pnatitl.replace("/", "_").replace(" ", "_").replace("&", "and")\
    .replace("(", "").replace(")", "").replace(",", "_").replace("-", "_").replace("'","").lower()
    
    sicd = SICD[index]
    
    if naics_dir_name not in os.listdir(base_year_loc): 
        naics_loc = base_year_loc + "/" + naics_dir_name
        os.mkdir(naics_loc)
        
        pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
        os.mkdir(pnatitl_loc)
        createGeoJSON(pnatitl_loc, sicd) 

        
    else: 
        naics_loc = base_year_loc + "/" + naics_dir_name
        
        if pnatitl_dir_name not in os.listdir(naics_loc):
            pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
            os.mkdir(pnatitl_loc)
            createGeoJSON(pnatitl_loc, sicd) 
          
        else: 
            pnatitl_loc = naics_loc + "/" + pnatitl_dir_name 
            createGeoJSON(pnatitl_loc, sicd) 
            
print datetime.now() - startTime

0:21:21.133813
